In [15]:
from corenlp_pywrap import pywrap
import pandas as pd
from pymongo import MongoClient
from functools import reduce
import numpy as np
import gensim

In [16]:
full_annotator_list = ["ssplit"]
cn = pywrap.CoreNLP(url='http://172.17.0.2:9000', annotator_list = full_annotator_list)

In [17]:
switchboard = pd.read_csv('datasets/switchboard.csv')
switchboard = switchboard.dropna(how='any', subset=['clean_text', 
                                                    'conversation_no'])
switchboard.dropna(how='any', subset=['clean_text', 'conversation_no']).isnull().values.any()
# print(switchboard[switchboard.act_label_1 == 'Backwards-Communicative-Function:Understanding:Sympathy'])


False

# Get the list of all mocs

In [18]:
mocs = np.unique(switchboard.act_label_1)
sup = [print(m) for m in mocs]

Backwards-Communicative-Function:Agreement:Accept
Backwards-Communicative-Function:Agreement:Accept-part
Backwards-Communicative-Function:Agreement:Hold-before-answer/agreement
Backwards-Communicative-Function:Agreement:Maybe
Backwards-Communicative-Function:Agreement:Reject
Backwards-Communicative-Function:Agreement:Reject-part
Backwards-Communicative-Function:Answer:Affirmative-non-yes-answers
Backwards-Communicative-Function:Answer:Dispreferred-answers
Backwards-Communicative-Function:Answer:Negative-non-no-answers
Backwards-Communicative-Function:Answer:No answers
Backwards-Communicative-Function:Answer:Other-answers
Backwards-Communicative-Function:Understanding:Acknowledge
Backwards-Communicative-Function:Understanding:Appreciation
Backwards-Communicative-Function:Understanding:Backchannel-in-question-form
Backwards-Communicative-Function:Understanding:Correct-misspeaking
Backwards-Communicative-Function:Understanding:Downplayer
Backwards-Communicative-Function:Understanding:Repe

# Print out general exampels

In [19]:
def get_action_example(m, idx=0, random=False):
    all_ms = list(switchboard[switchboard.act_label_1 == m]['clean_text'])
    if random:
        idx = np.random.randint(0, len(all_ms))
    return all_ms[idx]

In [20]:
for m in mocs:
    print("{}\n'{}'".format(m, get_action_example(m, random=True)))

Backwards-Communicative-Function:Agreement:Accept
'That's true.'
Backwards-Communicative-Function:Agreement:Accept-part
'Well, for some things.'
Backwards-Communicative-Function:Agreement:Hold-before-answer/agreement
'Uh ...'
Backwards-Communicative-Function:Agreement:Maybe
'That could be it, could be it.'
Backwards-Communicative-Function:Agreement:Reject
'Nay.'
Backwards-Communicative-Function:Agreement:Reject-part
'Um, um, well, n-'
Backwards-Communicative-Function:Answer:Affirmative-non-yes-answers
'It would be really nice because I'm kind of tired of driving in pot holes.'
Backwards-Communicative-Function:Answer:Dispreferred-answers
'Well, it's, this is going to sound really strange,'
Backwards-Communicative-Function:Answer:Negative-non-no-answers
'Uh, not anymore.'
Backwards-Communicative-Function:Answer:No answers
'No'
Backwards-Communicative-Function:Answer:Other-answers
'Well, s-, sort of.'
Backwards-Communicative-Function:Understanding:Acknowledge
'Yeah'
Backwards-Communicativ

# Identifying MOCs

In [7]:
def moc_filter(m, words):
    for w in words:
        if w.lower() in m.lower() :
            return True
    return False
def print_mocs(queries):
    for m in mocs:
        if moc_filter(m, queries):
            print("{}\n'{}'".format(m, get_action_example(m, random=True)))
def get_list(queries):
    return [m for m in mocs if moc_filter(m, queries)]
query_mocs = {}


## Ignore these phrases
These types of communication don't really fit in the common theme of shit we'll do.

In [8]:
# mocs to ignore
query_mocs['ignore'] = ['Communicative-Status:Non-verbal', 
                        'Communicative-Status:Uninterpretable',
                        'Communicative-Status:3rd-party-talk',
                        'Other:Transcription-errors', 
                        'Information-level:Task-management', 
                        'none',
                        'Other:Segment-(multi-utterance)',
                        'Other:Hedge',
                       'Other', 
                        'Other-forward-function:Explicit-performative', 
                       'Other-forward-function']
for m in query_mocs['ignore']:
    print(m)
    print('"%s"' %get_action_example(m, random=True))

Communicative-Status:Non-verbal
"."
Communicative-Status:Uninterpretable
"that"
Communicative-Status:3rd-party-talk
"And yourself."
Other:Transcription-errors
"place that, that needed this."
Information-level:Task-management
"Did you feel, uh, rewarded while you were doing it."
none
"it's like, oh, my"
Other:Segment-(multi-utterance)
"You know"
Other:Hedge
"Well, I hesitate to say we ought to spend ourselves to affluence"
Other
"And then"
Other-forward-function:Explicit-performative
"I will bet you those are Dutch iris."
Other-forward-function
"Well good luck with your expected baby there."


## Conventional parts of conversation MOC
Those that wrap the beginning and ending of a conversation. Something we'll probably encode in the symobllic dict


In [9]:
queries =  ['Conventional']
print_mocs(queries)
query_mocs['conventional'] = get_list(queries)

Other-forward-function:Conventional-closing
'Okay'
Other-forward-function:Conventional-opening
'this is Lois.'


## Answers to QA MOC

In [10]:
queries =  ['Backwards-Communicative-Function:Agreement:Hold-before-answer/agreement',
 'Backwards-Communicative-Function:Answer:Affirmative-non-yes-answers',
 'Backwards-Communicative-Function:Answer:Dispreferred-answers',
 'Backwards-Communicative-Function:Answer:Negative-non-no-answers',
 'Backwards-Communicative-Function:Answer:No answers',
 'Backwards-Communicative-Function:Answer:Other-answers',
 'Backwards-Communicative-Function:Understanding:Backchannel-in-question-form',
 'Info-request:Action-directive',
 'Info-request:Declarative-Wh-Question',
 'Info-request:Open-Question',
 'Info-request:Or-Clause',
 'Info-request:Or-Question',
 'Info-request:Wh-Question',
 'Info-request:Yes answers',
 'Info-request:Yes-No-Question']
print_mocs(queries)
query_mocs['qa'] = get_list(queries)

Backwards-Communicative-Function:Agreement:Hold-before-answer/agreement
'Well, I'm a drafter'
Backwards-Communicative-Function:Answer:Affirmative-non-yes-answers
'Oh, right,'
Backwards-Communicative-Function:Answer:Dispreferred-answers
'Well, there is a lake not far'
Backwards-Communicative-Function:Answer:Negative-non-no-answers
'Uh, not'
Backwards-Communicative-Function:Answer:No answers
'no'
Backwards-Communicative-Function:Answer:Other-answers
'Ooh, well I don't know.'
Backwards-Communicative-Function:Understanding:Backchannel-in-question-form
'Oh, really ?'
Info-request:Action-directive
'go ahead.'
Info-request:Declarative-Wh-Question
'like what ?'
Info-request:Open-Question
'Then how does he like it ?'
Info-request:Or-Clause
'or is this early admissions ?'
Info-request:Or-Question
'what, what do you advocate, more taxes or better management of what they got ?'
Info-request:Wh-Question
'what are the'
Info-request:Yes answers
'Uh-huh.'
Info-request:Yes-No-Question
'is this dinner t

## Generate Questions

In [11]:
queries =  [
 'Backwards-Communicative-Function:Understanding:Backchannel-in-question-form',
 'Info-request:Declarative-Wh-Question',
 'Info-request:Open-Question',
 'Info-request:Or-Clause',
 'Info-request:Or-Question',
 'Info-request:Wh-Question',
 'Info-request:Yes-No-Question']
print_mocs(queries)
query_mocs['answers'] = get_list(queries)

Backwards-Communicative-Function:Understanding:Backchannel-in-question-form
'Oh, yeah ?'
Info-request:Declarative-Wh-Question
'Whoever Troy Aikman is.'
Info-request:Open-Question
'How do you feel about that ?'
Info-request:Or-Clause
'or, how do you work your budgeting ?'
Info-request:Or-Question
'Are you, uh, shooting for a bachelor's now, or a master's, or what ?'
Info-request:Wh-Question
'what is it, uh, the act where they had to cut everything across the board'
Info-request:Yes-No-Question
'are you on an exercise program now'


## General Dialogue

In [12]:
queries =  ['Rhetorical-Questions']
print_mocs(queries)
query_mocs['dialog'] = get_list(queries)

Rhetorical-Questions
'but hey, what can you do.'


## Understanding
Signals that you understand or completely reject. 

In [13]:
# queries = ['Understanding', 'Agreement']
queries = ['Backwards-Communicative-Function:Agreement:Accept',
 'Backwards-Communicative-Function:Agreement:Accept-part',
 'Backwards-Communicative-Function:Agreement:Hold-before-answer/agreement',
 'Backwards-Communicative-Function:Agreement:Maybe',
 'Backwards-Communicative-Function:Agreement:Reject',
 'Backwards-Communicative-Function:Agreement:Reject-part',
 'Backwards-Communicative-Function:Understanding:Acknowledge',
 'Backwards-Communicative-Function:Understanding:Appreciation',
 'Backwards-Communicative-Function:Understanding:Backchannel-in-question-form',
 'Backwards-Communicative-Function:Understanding:Downplayer',
 'Backwards-Communicative-Function:Understanding:Repeat-phrase',
 'Backwards-Communicative-Function:Understanding:Response Acknowledgement',
 'Backwards-Communicative-Function:Understanding:Sympathy']
print_mocs(queries)
query_mocs['understanding'] = get_list(queries)

Backwards-Communicative-Function:Agreement:Accept
'Yeah'
Backwards-Communicative-Function:Agreement:Accept-part
'he-, he-, he's not bad'
Backwards-Communicative-Function:Agreement:Hold-before-answer/agreement
'Well, let's see'
Backwards-Communicative-Function:Agreement:Maybe
'I think so.'
Backwards-Communicative-Function:Agreement:Reject
'Well, no'
Backwards-Communicative-Function:Agreement:Reject-part
'I, I don't disagree totally with that'
Backwards-Communicative-Function:Understanding:Acknowledge
'Uh-huh.'
Backwards-Communicative-Function:Understanding:Appreciation
'My goodness.'
Backwards-Communicative-Function:Understanding:Backchannel-in-question-form
'He's one of the two, huh.'
Backwards-Communicative-Function:Understanding:Downplayer
'That's okay'
Backwards-Communicative-Function:Understanding:Repeat-phrase
'Twelve and sixteen.'
Backwards-Communicative-Function:Understanding:Response Acknowledgement
'Oh.'
Backwards-Communicative-Function:Understanding:Sympathy
'Oh, I'm so sorry

## Clarification
Questions asked when things aren't clear and the repsonses to be made when those things are not clear

In [14]:
queries=['Backwards-Communicative-Function:Understanding:Correct-misspeaking',
         'Backwards-Communicative-Function:Understanding:Signal-non-understanding',
         'Backwards-Communicative-Function:Understanding:Repeat-phrase',
         'Backwards-Communicative-Function:Understanding:Summarize/reformulate']
print_mocs(queries)
query_mocs['clarification'] = get_list(queries)

Backwards-Communicative-Function:Understanding:Correct-misspeaking
'Midnight.'
Backwards-Communicative-Function:Understanding:Repeat-phrase
'yeah'
Backwards-Communicative-Function:Understanding:Signal-non-understanding
'Locust Hill ?'
Backwards-Communicative-Function:Understanding:Summarize/reformulate
'You were helping your parents.'


## Kindness

In [15]:
queries = ["Other-forward-function:Apology", "Other-forward-function:Thanking", "Other-forward-function:You're-Welcome"]

print_mocs(queries)
query_mocs['kindness'] = get_list(queries)

Other-forward-function:Apology
'Sorry I roped on Volvos then.'
Other-forward-function:Thanking
'Well, thank you.'
Other-forward-function:You're-Welcome
'Uh-huh.'


## Opinions / Personality

In [14]:
queries = ['Statement-opinion', 'Communicative-Status:Self-talk', 'Forward-Communicative-Function:Statement:Statement-non-opinion', 'Other-forward-function:Exclamation']
print_mocs(queries)
query_mocs['self'] = get_list(queries)


NameError: name 'print_mocs' is not defined

## Persuasion / Direction


In [30]:
queries = ['Committing-speaker-future-action:Commit', 'Committing-speaker-future-action:Offer', 'Influencing-addressee-fut-actn:Open-option', 'Rhetorical-Questions','Info-request:Action-directive',]
print_mocs(queries)
query_mocs['influence'] = get_list(queries)


Committing-speaker-future-action:Commit
'Let me push one here.'
Committing-speaker-future-action:Offer
'let me look at my husband and ask him'
Influencing-addressee-fut-actn:Open-option
'We could talk about my favorite subject.'
Info-request:Action-directive
'Well, tell me this, uh, Lowell'
Rhetorical-Questions
'isn't that funny.'


# Checking to make sure that all the queries are covered

In [18]:
mocs_remaining = list(mocs)
for query_type, query_list in query_mocs.items():
    print('Evaluating moc "{}"'.format(query_type))
    for q in query_list:
        if q not in mocs_remaining:
            print('\t "%s" already caught by a previous moc' %q)
        else:
            mocs_remaining.remove(q)
if len(mocs_remaining) > 0:
    print("these are the remaining mocs")
    _ = [print('\t',m) for m in mocs_remaining]

Evaluating moc "conventional"
Evaluating moc "kindness"
Evaluating moc "clarification"
Evaluating moc "dialog"
Evaluating moc "influence"
	 "Rhetorical-Questions" already caught by a previous moc
Evaluating moc "qa"
	 "Info-request:Action-directive" already caught by a previous moc
Evaluating moc "ignore"
Evaluating moc "answers"
	 "Backwards-Communicative-Function:Understanding:Backchannel-in-question-form" already caught by a previous moc
	 "Info-request:Declarative-Wh-Question" already caught by a previous moc
	 "Info-request:Open-Question" already caught by a previous moc
	 "Info-request:Or-Clause" already caught by a previous moc
	 "Info-request:Or-Question" already caught by a previous moc
	 "Info-request:Wh-Question" already caught by a previous moc
	 "Info-request:Yes-No-Question" already caught by a previous moc
Evaluating moc "understanding"
	 "Backwards-Communicative-Function:Agreement:Hold-before-answer/agreement" already caught by a previous moc
	 "Backwards-Communicative-

## Examples of remaining mocs

In [19]:
for m in mocs_remaining:
    print("{}\n'{}'".format(m, get_action_example(m, random=True)))

In [31]:
test = "Influencing-addressee-fut-actn:Open-option"

for i in range(10):
    print(get_action_example(test, i))

You can go first
or I will.
well, we could talk about that
and then we could talk a bit about, um, just quality of products in general
And I guess, the suggestion is that we maybe talk about a menu for a dinner party, if we wanted to do something like that or
We could talk about my favorite subject.


IndexError: list index out of range

In [ ]:
# influencing the future
['Committing-speaker-future-action:Commit',
'Committing-speaker-future-action:Offer',
'Influencing-addressee-fut-actn:Open-option']


In [31]:
sup_then=  \
"declaration": {
    "opinionated": "I think vegetarian parents should give children the choice to be vegetarian or not.",
    "fact / certain declaration" : "The capital of Alabama is Montegomery.",
    "uncertain declaration" : "I think Hemingway's cats might have had 6 toes on each paw.",
    "exclamation": "Wow.",
    "offer": "I can inform the president, if you want.",
    "future-commitment": "I'm gonna tell my wife about this chat.",
},
"understanding": {
    "appreciation": "Thanks so much for the advice",
    "apology": "My bad.",
    "acknowledgement": "I know what you're talking about ",
    "full disagreement" : "The world doesn't work that way.",
    "partial disagreement": "I don't think you're right, but I'm not sure",
    "partial agreement": "I guess you're right, but I think it's possible they may have thought of this before",
    "full agreement": "Absolutely true.",

},
"question": {
    "Context based question": "Did you ever find the dog you were looking for?",
    "Ask for Opinion": "What do you think about 80s music?",
    "Clarifying Question": "What do you mean you jumped over the grand canyon?",
    "Factoid Question": "What's the capital of Alabama?"
},
"answer": {
    "factoid based answer": "The capital of Alabama is Montegomery.",
    "opinion based answer": "I think we should give free coffee to tired people.",
    "i dont know answer": "I'm not really not sure",
    "maybe": "That may be the case, but I'm not sure",

    "full disagreement": "Nope",
    "partial disagreement": "I don't think you're right, but I'm not sure",
    "partial agreement": "I guess you're right, but I think it's possible they may have thought of this before",
    "full agreement": "Absolutely true.",
    "other answer": ""
},
"narrative" : {
    "start story" : "One time I was spelunking in Santa Cruz",
    "narrative content" : "While I was there, I found a spider the size of my hand",
},
"conversational_formality": {
    "closing": "Goodbye",
    "courtesies": [
        "Thank you",
        "You're welcome!"
    ],
    "opening": "Hi there!"
},
"empathy": {
    "sympathy": "I'm so sorry for your loss",
    "relating": "Wow I also like to play the Banjo",
    "compliment": "You sound like you're doing so well"
},


"influence": {
    "Rhetorical-Question": "Can you blame your dog for wanting attention?",
    "steer conversation": "I'm gonna change the subject to Baseball",
    "suggestion": "Don't you think you should listen to what your kids say?",
    "command": "Stop telling your dog to jump through hoops!"
},

"identity": {
    "identifying information": [
        "I am thirty years old",
        "I was born in a factory"
    ],
    "referene to past" : "I once backed my car into a trash can.",
},

"clarification": {
    "clarifying question": "What do you mean you jumped over the grand canyon?",
    "correct misspeaking": "I meant to say that ",
    "Summarize/reformulate": "I'll give my dog treats whenever it tries to jump",
    "Signal non-understanding": "Um, I'm not sure what you mean by that.",
    "Repeat phrase": "I said, I'll teach my dog how to jump through the hoop."
},

"humor": {
    "rhetorical question": "Can you blame your dog for wanting attention?",
    "wit/wordplay": "Time flies like an arrow; fruit flies like a banana.",
    "pun": "I'm reading a book about anti-gravity. It's impossible to put down.",
    "joke": "I dreamt I was forced to eat a giant marshmallow. When I woke up, my pillow was gone.",
    "sarcasm": "I totally want to be run over by a car"
},

"other": {
    "filler word": "Um."
},

SyntaxError: invalid syntax (<ipython-input-31-435da2dd8294>, line 1)

In [37]:
import json
# print(json.dumps(sup_then, sort_keys=True,
#                   indent=4, separators=(',', ': ')))
[print(p) for p in sorted(sup_then.keys())]

answer
clarification
conversational_formality
declaration
empathy
humor
identity
influence
other
question
understanding


[None, None, None, None, None, None, None, None, None, None, None]